In [1]:
#V5 讓Code幫我複製貼上到海外債券一覽表

import datetime
import pandas as pd

# 取得今天日期，並轉換為字串 'yyyymmdd' 格式
today = datetime.date.today().strftime('%Y%m%d')

# 設定檔案路徑，檔名為今天日期_TraderPosition.xlsx
file_path = f'data/{today}_TraderPosition.xlsx'

# 有哪些人
names = ['Jesse', 'Aaron', 'Mark1', 'Gary', 'PETE']

# 要讀取的工作表名稱
sheet_names = names

# 將每個工作表的 DataFrame 存入以工作表名稱為 key 的字典中
df_dict = {}
for sheet_name in sheet_names:
    df = pd.read_excel(file_path, sheet_name=sheet_name, usecols='A:I')
    start_row = df.loc[df.iloc[:, 0] == "ProductID"].index
    end_row = df.loc[df.iloc[:, 0] == 'total'].index

    if start_row.empty or end_row.empty:
        # 如果沒有找到 ProductID 或 total，則存入一個空的 DataFrame
        df = pd.DataFrame(columns=['ProductID', 'Trade Date', 'closed date', 
                                   'closed ticket no.', 'Lots', 'Broker', 
                                   'DV01', 'Ticket Number', 'Cost'])
        df['Cost'] = pd.Series(dtype='float64')
    else:
        start_row = start_row[0] + 1
        end_row = end_row[0]
        df = df.iloc[start_row:end_row, :]
        df = df[~df.iloc[:, 0].astype(str).str.contains('total', case=False)]

    # 重新設定 column 名稱
    df.columns = ['ProductID', 'Trade Date', 'closed date', 
                  'closed ticket no.', 'Lots', 'Broker', 
                  'DV01', 'Ticket Number', 'Cost']

    # 對每個 DataFrame 的 row 重新編號
    df = df.reset_index(drop=True)

    # 將 DataFrame 存入變數 df_{sheet_name} 中
    var_name = f'df_{sheet_name}'
    globals()[var_name] = df
    df_dict[sheet_name] = df

#print(df_dict)


In [2]:
import pandas as pd
import datetime

# 取得今天日期，並轉換為字串 'yymmdd' 格式
today = datetime.date.today().strftime('%y%m%d')

# 設定檔案路徑，檔名為海外債券一覽表{today}.xlsm
file_path = f'data/海外債券一覽表{today}.xlsm'

# 根據人名選擇要讀取的工作表名稱
sheet_names = [f'{name}_{suffix}' for name in names for suffix in ['UR', 'Real']]

# 讀取Excel檔案中指定的工作表，並存入以工作表名稱為 key 的字典中
po_dict = {}
for sheet_name in sheet_names:
    po = pd.read_excel(file_path, sheet_name=sheet_name, usecols="A:I", header=0)
    start_row = po.loc[po.iloc[:, 0] == "USD_Futures"].index[0] + 1 #從USD_Futures底下開始
    end_row = po.loc[po.iloc[:, 0] == "USD_total"].index[0]
    po = po.iloc[start_row:end_row, :] #到USD_Total上方結尾


    # 將 USD_Futures 那一列的資料轉換成對應的 column name
    column_names = list(po.iloc[0])
    po.columns = column_names
    po = po.iloc[1:, :]
    po = po.reset_index(drop=True)  # 重新編號 row index
    po_dict[sheet_name] = po

    # 將 DataFrame 存入變數 po_{sheet_name} 中
    var_name = f'po_{sheet_name}'
    globals()[var_name] = po

    po_dict[sheet_name].dropna(axis=0, how='all', inplace=True)

#print(po_dict)

# 創建一個空的 dataframe，列名為姓名，列數為0
for name in names:
    # 使用 po_dict[f'{name}_Real'].columns 以保持相同的列名
    globals()[f'po_{name}_Real_New'] = pd.DataFrame(columns=po_dict[f'{name}_Real'].columns)

    # 為每個 po_人名_Real_New dataframe 新增一個 'MKT Price' 列
    globals()[f'po_{name}_Real_New']['MKT Price'] = pd.Series(dtype='float64')

In [3]:
def process_conditions(df_name, po_name_UR, po_name_Real_New):
    while True:
        condition1_met = False
        for index, row in df_name.iterrows():
            product_id = row['ProductID']
            broker = row['Broker']
            matching_rows = po_name_UR[(po_name_UR['ProductID'] == product_id) & (po_name_UR['Broker'] == broker)]
            for ur_index, matching_row in matching_rows.iterrows():
                if matching_row['Lots'] != 0:
                    # 情況一
                    if row['Lots'] > 0 and matching_row['Lots'] < 0 and abs(row['Lots']) >= abs(matching_row['Lots']):
                        matching_row['MKT Price'] = row['Cost']  # 更新 MKT Price 欄位
                        po_name_Real_New = pd.concat([po_name_Real_New, pd.DataFrame([matching_row])], ignore_index=True)
                        df_name.at[index, 'Lots'] = row['Lots'] - abs(matching_row['Lots'])  # 更新 df_人名 的 Lots 欄位
                        po_name_UR.at[ur_index, 'Lots'] = 0  # 更新 po_人名_UR 的 Lots 欄位
                        condition1_met = True
                        break
        if not condition1_met:
            break

    while True:
        condition2_met = False
        for index, row in df_name.iterrows():
            product_id = row['ProductID']
            broker = row['Broker']
            matching_rows = po_name_UR[(po_name_UR['ProductID'] == product_id) & (po_name_UR['Broker'] == broker)]
            for ur_index, matching_row in matching_rows.iterrows():
                if matching_row['Lots'] != 0:
                    # 情況二
                    if row['Lots'] > 0 and matching_row['Lots'] < 0 and abs(row['Lots']) < abs(matching_row['Lots']):
                        new_row = matching_row.copy()
                        new_row['Lots'] = -row['Lots']  # 更新 Lots 欄位
                        new_row['MKT Price'] = row['Cost']  # 更新 MKT Price 欄位
                        po_name_Real_New = pd.concat([po_name_Real_New, pd.DataFrame([new_row])], ignore_index=True)
                        po_name_UR.at[ur_index, 'Lots'] = matching_row['Lots'] + row['Lots']  # 更新 po_人名_UR 的 Lots 欄位
                        df_name.at[index, 'Lots'] = 0  # 更新 df_人名 的 Lots 欄位
                        condition2_met = True
                        break
        if not condition2_met:
            break

    while True:
        condition3_met = False
        for index, row in df_name.iterrows():
            product_id = row['ProductID']
            broker = row['Broker']
            matching_rows = po_name_UR[(po_name_UR['ProductID'] == product_id) & (po_name_UR['Broker'] == broker)]
            for ur_index, matching_row in matching_rows.iterrows():
                if matching_row['Lots'] != 0:
                    # 情況三
                    if row['Lots'] < 0 and matching_row['Lots'] > 0 and abs(row['Lots']) >= abs(matching_row['Lots']):
                        matching_row['MKT Price'] = row['Cost']  # 更新 MKT Price 欄位
                        po_name_Real_New = pd.concat([po_name_Real_New, pd.DataFrame([matching_row])], ignore_index=True)
                        df_name.at[index, 'Lots'] = row['Lots'] + matching_row['Lots']  # 更新 df_人名 的 Lots 欄位
                        po_name_UR.at[ur_index, 'Lots'] = 0  # 更新 po_人名_UR 的 Lots 欄位
                        condition3_met = True
                        break
        if not condition3_met:
            break

    while True:
        condition4_met = False
        for index, row in df_name.iterrows():
            product_id = row['ProductID']
            broker = row['Broker']
            matching_rows = po_name_UR[(po_name_UR['ProductID'] == product_id) & (po_name_UR['Broker'] == broker)]
            for ur_index, matching_row in matching_rows.iterrows():
                if matching_row['Lots'] != 0:
                    # 情況四
                    if row['Lots'] < 0 and matching_row['Lots'] > 0 and abs(row['Lots']) < abs(matching_row['Lots']):
                        new_row = matching_row.copy()
                        new_row['Lots'] = row['Lots']  # 更新 Lots 欄位
                        new_row['MKT Price'] = row['Cost']  # 更新 MKT Price 欄位
                        po_name_Real_New = pd.concat([po_name_Real_New, pd.DataFrame([new_row])], ignore_index=True)
                        po_name_UR.at[ur_index, 'Lots'] = matching_row['Lots'] + row['Lots']  # 更新 po_人名_UR 的 Lots 欄位
                        df_name.at[index, 'Lots'] = 0  # 更新 df_人名 的 Lots 欄位
                        condition4_met = True
                        break
        if not condition4_met:
            break

    # 情況五
    for index, row in df_name.iterrows():
        if row['Lots'] != 0:
            matching_rows = po_name_UR[(po_name_UR['Lots'] == 0)]
            for ur_index, matching_row in matching_rows.iterrows():
                po_name_UR.loc[ur_index] = row  # 使用 .loc 來更新 po_人名_UR 的整行資料
                df_name.at[index, 'Lots'] = 0  # 更新 df_人名 的 Lots 欄位
                break  # 當找到匹配的行並完成更新後，跳出迴圈
         
                
    return df_name, po_name_UR, po_name_Real_New



for name in names:
    df_name = globals()[f'df_{name}']
    po_name_UR = globals()[f'po_{name}_UR']
    po_name_Real_New = globals()[f'po_{name}_Real_New']

    # 大前提和所有情況
    df_name, po_name_UR, po_name_Real_New = process_conditions(df_name, po_name_UR, po_name_Real_New)

    globals()[f'po_{name}_Real_New'] = po_name_Real_New


# 將 df_人名 中 Lots 非 0 的 row 貼到 po_人名_UR 的底下
for name in names:
    df_name = globals()[f'df_{name}']
    po_name_UR = globals()[f'po_{name}_UR']

    # 將 df_人名 中 Lots 非 0 的 row 貼到 po_人名_UR 的底下
    remaining_rows = df_name[df_name['Lots'] != 0]
    po_name_UR = pd.concat([po_name_UR, remaining_rows], ignore_index=True)
    df_name['Lots'] = 0  # 將 df_人名 中的 Lots 欄位設為 0

    globals()[f'po_{name}_UR'] = po_name_UR

In [4]:
#上面是對從df_人名到po_人名_UR以及po_人名_Real_New的code
#這個是對po_人名_UR自己的code

def process_conditions_v2(df_name, po_name_UR, po_name_Real_New):
    condition_met = True
    while condition_met:
        condition_met = False
        for index, row1 in po_name_UR.iterrows():
            if row1['Lots'] != 0:
                product_id = row1['ProductID']
                broker = row1['Broker']
                matching_rows = po_name_UR[(po_name_UR['ProductID'] == product_id) & (po_name_UR['Broker'] == broker)]
                for ur_index, row2 in matching_rows.iterrows():
                    if row2['Lots'] != 0:
                        # 情況一
                        if row1['Lots'] > 0 and row2['Lots'] < 0 and abs(row1['Lots']) >= abs(row2['Lots']):
                            row3 = row2.copy()
                            row3['MKT Price'] = row1['Cost']  # 更新 MKT Price 欄位
                            po_name_Real_New = pd.concat([po_name_Real_New, row3.to_frame().T], ignore_index=True)
                            po_name_UR.at[index, 'Lots'] = row1['Lots'] - abs(row2['Lots'])  # 更新 po_人名_UR 的 Lots 欄位
                            po_name_UR.at[ur_index, 'Lots'] = 0  # 更新 row2 的 Lots 欄位
                            condition_met = True
                            break
                        # 情況二
                        elif row1['Lots'] > 0 and row2['Lots'] < 0 and abs(row1['Lots']) < abs(row2['Lots']):
                            row3 = row2.copy()
                            row3['Lots'] = -row1['Lots']  # 更新 Lots 欄位
                            row3['MKT Price'] = row1['Cost']  # 更新 MKT Price 欄位
                            po_name_Real_New = pd.concat([po_name_Real_New, row3.to_frame().T], ignore_index=True)
                            po_name_UR.at[ur_index, 'Lots'] = row2['Lots'] + row1['Lots']  # 更新 row2 的 Lots 欄位
                            po_name_UR.at[index, 'Lots'] = 0  # 更新 row1 的 Lots 欄位
                            condition_met = True
                            break
                        # 情況三
                        elif row1['Lots'] < 0 and row2['Lots'] > 0 and abs(row1['Lots']) >= abs(row2['Lots']):
                            row3 = row2.copy()
                            row3['MKT Price'] = row1['Cost']  # 更新 MKT Price 欄位
                            po_name_Real_New = pd.concat([po_name_Real_New, row3.to_frame().T], ignore_index=True)
                            po_name_UR.at[index, 'Lots'] = row1['Lots'] + row2['Lots']  # 更新 row1 的 Lots 欄位
                            po_name_UR.at[ur_index, 'Lots'] = 0  # 更新 row2 的 Lots 欄位
                            condition_met = True
                            break
                        # 情況四
                        elif row1['Lots'] < 0 and row2['Lots'] > 0 and abs(row1['Lots']) < abs(row2['Lots']):
                            row3 = row2.copy()
                            row3['Lots'] = -row1['Lots']  # 更新 Lots 欄位
                            row3['MKT Price'] = row1['Cost']  # 更新 MKT Price 欄位
                            po_name_Real_New = pd.concat([po_name_Real_New, row3.to_frame().T], ignore_index=True)
                            po_name_UR.at[ur_index, 'Lots'] = row2['Lots'] + row1['Lots']  # 更新 row2 的 Lots 欄位
                            po_name_UR.at[index, 'Lots'] = 0  # 更新 row1 的 Lots 欄位
                            condition_met = True
                            break
    return po_name_UR, po_name_Real_New


for name in names:
    po_name_UR = globals()[f'po_{name}_UR']
    po_name_Real_New = globals()[f'po_{name}_Real_New']

    # 執行新的大前提和情況
    po_name_UR, po_name_Real_New = process_conditions_v2(df_name, po_name_UR, po_name_Real_New)

    globals()[f'po_{name}_Real_New'] = po_name_Real_New

    
#將po_UR的DV01調整
def update_dv01(row):
    product_id = row['ProductID']
    lots = row['Lots']
    if product_id.startswith('TU'):
        return -18.6 * 2 * lots
    elif product_id.startswith('FV'):
        return -42.7 * lots
    elif product_id.startswith('TY'):
        return -77 * lots
    elif product_id.startswith('US'):
        return -198 * lots
    else:
        return row['DV01']

for name in names:
    po_name_UR = globals()[f'po_{name}_UR']
    po_name_UR['DV01'] = po_name_UR.apply(update_dv01, axis=1)
    globals()[f'po_{name}_UR'] = po_name_UR
    




In [5]:
#將po_Real_New新增欄位MTM P&L，並用公式計算
def calculate_mtm_pnl(row):
    if row['ProductID'][:2] == "TU":
        return (row['MKT Price'] - row['Cost']) * row['Lots'] * 2000
    else:
        return (row['MKT Price'] - row['Cost']) * row['Lots'] * 1000
    
    
for name in names:
    po_name_Real_New = globals()[f'po_{name}_Real_New']
    
    if po_name_Real_New.empty:
        po_name_Real_New['MTM P&L'] = pd.Series(dtype='float64')
    else:
        po_name_Real_New['MTM P&L'] = po_name_Real_New.apply(calculate_mtm_pnl, axis=1)

    globals()[f'po_{name}_Real_New'] = po_name_Real_New

# 將所有 DataFrame 寫入一個新的 Excel 檔案中，並將df_人名工作表放在所有其他工作表之後。
output_file_path = f'data/{today}_TraderPosition_Output.xlsx'

with pd.ExcelWriter(output_file_path) as writer:
    for name in names:
        po_name_UR = globals()[f'po_{name}_UR']
        po_name_Real_New = globals()[f'po_{name}_Real_New']

        po_name_UR.to_excel(writer, sheet_name=f'po_{name}_UR', index=False)
        po_name_Real_New.to_excel(writer, sheet_name=f'po_{name}_Real_New', index=False)

    for name in names:
        df_name = globals()[f'df_{name}']
        df_name.to_excel(writer, sheet_name=f'df_{name}', index=False)

In [6]:
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows

# 載入工作簿
filepath = f"data/海外債券一覽表{today}.xlsm"
wb = openpyxl.load_workbook(filepath)

for name in names:
    # 選擇工作表
    ws = wb[f'{name}_UR']

    # 定位範圍
    for row in ws.iter_rows():
        if row[0].value == 'USD_Futures':
            start_row = row[0].row + 2
        elif row[0].value == 'USD_total':
            end_row = row[0].row - 1
            break

    # 檢查範圍是否足夠
    po_name_UR = globals()[f'po_{name}_UR']
    if end_row - start_row + 1 < len(po_name_UR):
        print(f"{name} 超過了!")
        # 計算需要插入的行數
        rows_to_insert = len(po_name_UR) - (end_row - start_row + 1)
        # 在指定位置插入行
        ws.insert_rows(end_row, rows_to_insert)
        # 更新結束行
        end_row += rows_to_insert

    # 貼上數據
    for index, row in enumerate(dataframe_to_rows(po_name_UR, index=False, header=False), start_row):
        for col_num, value in enumerate(row, 1):
            ws.cell(row=index, column=col_num, value=value)

# 保存工作簿
wb.save(filepath)



Jesse 超過了!
